In [60]:
import tensorflow as tf

In [69]:
a = tf.constant([1.0, 2.0], name='a', dtype=tf.float32)  # 定义常量向量
b = tf.constant([2.0, 3.0], name='b')
result = a + b  # 向量相加
print(result)

# 先生成一个会话，通过该会话来计算结果
# sess = tf.Session()
sess = tf.InteractiveSession()  # 自动将生成的会话注册为默认会话
print(sess.run(result))
print(result.eval(session=sess))
sess.close()  # 关闭会话，释放资源

Tensor("add_20:0", shape=(2,), dtype=float32)
[3. 5.]
[3. 5.]


### 计算模型

- TensorFlow中的所有计算都会被转化为计算图上的节点。
- 而节点之间的边描述了计算之间的依赖关系。
- 在TensorFlow中，张量可以被简单地理解为多维数组。
- TensorFlow是一个通过计算图的形式来表述计算的编程系统。

In [62]:
g1 = tf.Graph()  # 生成新的计算图
with g1.as_default():
    # 在计算图g1中定义变量'v'，并设置初始值为0
    v = tf.get_variable("v", initializer=tf.zeros(shape=[1]))
    
# 在计算图g1中读取变量'v'的值
with tf.Session(graph=g1) as sess:  # 通过上下文管理器来使用会话
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print(sess.run(tf.get_variable('v')))

[0.]


### 数据模型

- 在张量中，并没有真正保存数字，它保存的是如何得到这些数字的计算过程。
- 一个张量中主要保存了三个属性：名字、维度、类型。
- 张量的命名可通过"node:src_output"的形式给出。其中，node为节点名称，src_output表示当前张量来自节点的第几个输出。
- TensorFlow支持的类型主要包括：tf.float32, tf.float64, tf.int8, tf.int16, tf.int32, tf.int64, tf.uint8, tf.bool, tf.complex64, tf.complex128

### 运行模型

In [63]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)

### TensorFlow及神经网络

神经网络解决分类问题的主要步骤：
- 提取问题中实体的特征向量作为神经网络的输入。
- 定义神经网络的结构，并定义如何从神经网络的输入得到输出。
- 通过训练数据来调整神经网络中参数的取值，这就是训练神经网络的过程。
- 使用训练好的神经网络来预测未知的数据。

全连接神经网络：相邻两层之间任意两个节点之间都有连接。

TensorFlow支持的随机数生成函数：
- `tf.random_normal`：正态分布。
- `tf.truncated_normal`：正态分布，但若随机值偏离平均值超过2个标准差，将被重新随机。
- `tf.random_uniform`：平均分布。
- `tf.random_gamma`：Gamma分布。

TensorFlow常数生成函数：
- `tf.zeros([2,3], int32)`：产生全0的数组。
- `tf.ones([2,3], int32)`：产生全1的数组。
- `tf.fill([2,3], 9)`：产生一个全部为给定数字的组合。
- `tf.constant([1,2,3)`：产生一个给定值的常量。

In [64]:
# 声明一个2*3的矩阵变量，并赋予均值为0，标准差为2的随机数
weigths = tf.Variable(tf.random_normal([2, 3], mean=0, stddev=2))
biases = tf.Variable(tf.zeros([3]))

In [65]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))  # 该运算的输出结果即为张量
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))
x = tf.constant([[0.7, 0.9]])  # 1*2的矩阵

a = tf.matmul(x, w1)  # 矩阵乘法
y = tf.matmul(a, w2)

with tf.Session() as sess:
    # sess.run(w1.initializer)  # 逐个初始化变量
    # sess.run(w2.initializer)
    # 初始化所有变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(y))
    print(tf.all_variables)

[[3.957578]]
<function all_variables at 0x000000001B7790D0>


- `tf.all_variables`：可拿到当前计算图上所有的变量。
- `tf.trainable_variables`：得到所有需要优化的参数。
- 变量的类型是不可改变的。
- 维度在程序运行中是有可能改变的，需通过设置参数`validate_shape=False`。

In [66]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1)) 
w2 = tf.Variable(tf.random_normal([2, 2], stddev=1, seed=1))

# tf.assign(w1, w2)  # wrong
tf.assign(w1, w2, validate_shape=False)

<tf.Tensor 'Assign_2:0' shape=(2, 2) dtype=float32_ref>

In [67]:
# 使用placeholder实现前向传播算法
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 输入为n*2矩阵，前向传播结果为n*1的矩阵
# placeholder中数据的维度信息可以根据提供的数据推导得出，所有不一定要给出
x = tf.placeholder(tf.float32, shape=(3, 2), name='input') 
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# print(sess.run(y))  # 某个需要的placeholder没有被指定取值，报错
print(sess.run(y, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]}))  # 指定x的取值

[[3.957578 ]
 [1.1537654]
 [3.1674924]]


In [74]:
from numpy.random import RandomState

batch_size = 8

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 定义损失函数和反向传播的算法
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

# 创建会话
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 5000  # 训练的轮数
    for i in range(STEPS):
        # 每次选取batch_size个样本进行训练
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        # 根据样本训练神经网络并更新参数
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        
        if i % 1000 == 0:
            # 每个一定轮数，计算在所有数据上的交叉熵
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print("After %d training step(s), cross entropy on all data is %g" % (i, total_cross_entropy))
            
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 0.0674925
After 1000 training step(s), cross entropy on all data is 0.0163385
After 2000 training step(s), cross entropy on all data is 0.00907547
After 3000 training step(s), cross entropy on all data is 0.00714436
After 4000 training step(s), cross entropy on all data is 0.00578471
[[-1.9618274  2.582354   1.6820377]
 [-3.4681718  1.0698233  2.11789  ]]
[[-1.8247149]
 [ 2.6854665]
 [ 1.418195 ]]


训练神经网络的过程：
- 定义神经网络的结构和前向传播算法的输出结果；
- 定义损失函数以及选择反向传播优化的算法；
- 生成会话，并且在训练数据上反复运行反向传播优化算法。